In [477]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [478]:
#colums of previous df are messed up during EDA so crearting again
df = pd.read_csv('train.csv')  #parse_dates=True

In [479]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [480]:
X = df.drop(['count', 'casual', 'registered'], axis=1) 
y = df['count']
#drop 'casual', 'registered here because test.csv doesn't have these columns

In [481]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [482]:
def extract(df):
        df.datetime = pd.to_datetime(df.datetime)
        df['day'] = df.datetime.dt.day
        df['hour'] = df.datetime.dt.hour
        df['weekday'] = df.datetime.dt.weekday
        df['year'] = df.datetime.dt.year
        df['month'] = df.datetime.dt.month
        
        return pd.concat([df[['day']],pd.get_dummies(df[['year','month','weekday','hour']],columns=['year','month','weekday','hour'])],axis=1)



In [483]:
preprosessor = ColumnTransformer([
    ('do_nothing', 'passthrough', ['holiday', 'workingday', 'season', 'weather', 'windspeed', 'temp', 'humidity']),
    ('time_extact', FunctionTransformer(extract), ['datetime']),
    #('one_hot_encoding', OneHotEncoder(sparse = False), ['season','weather']),
    #('0_imputer', SimpleImputer(strategy='mean', fill_value=0), ['windspeed']),
    ], 
    remainder='drop')  

In [484]:
# create the model pipeline
pipeline = make_pipeline(preprosessor, RandomForestRegressor())

In [485]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                                  ['holiday', 'workingday',
                                                   'season', 'weather',
                                                   'windspeed', 'temp',
                                                   'humidity']),
                                                 ('time_extact',
                                                  FunctionTransformer(func=<function extract at 0x7fa61f6e6af0>),
                                                  ['datetime'])])),
                ('randomforestregressor', RandomForestRegressor())])

In [486]:
X_train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
2930,2011-07-11 00:00:00,3,0,1,1,28.70,32.575,65,12.9980
7669,2012-05-18 22:00:00,2,0,1,1,22.96,26.515,52,22.0028
1346,2011-04-01 23:00:00,2,0,1,1,12.30,15.910,61,6.0032
9432,2012-09-16 09:00:00,3,0,0,1,23.78,27.275,60,8.9981
453,2011-02-01 23:00:00,1,0,1,3,8.20,9.850,93,12.9980
...,...,...,...,...,...,...,...,...,...
5734,2012-01-14 02:00:00,1,0,0,1,6.56,8.335,47,11.0014
5191,2011-12-10 09:00:00,4,0,0,1,11.48,12.880,61,19.0012
5390,2011-12-18 16:00:00,4,0,0,1,11.48,13.635,48,16.9979
860,2011-02-19 07:00:00,1,0,0,1,15.58,19.695,17,35.0008


In [487]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8164 entries, 2930 to 7270
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    8164 non-null   object 
 1   season      8164 non-null   int64  
 2   holiday     8164 non-null   int64  
 3   workingday  8164 non-null   int64  
 4   weather     8164 non-null   int64  
 5   temp        8164 non-null   float64
 6   atemp       8164 non-null   float64
 7   humidity    8164 non-null   int64  
 8   windspeed   8164 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 637.8+ KB


In [488]:
pipeline.score(X_train, y_train)

0.9884566445874494

In [489]:
pipeline.score(X_test, y_test)

0.9269273833258622

## Cross Validation

In [490]:
pipeline = make_pipeline(preprosessor, RandomForestRegressor()) #an unfitted model, default value

In [491]:
cross_acc = cross_val_score(estimator=pipeline,        # estimator: # the model you want to evaluate/unfitted 
                            X=X_train,            # the training input data 
                            y=y_train,            # the training output data  
                            cv=5,                # number of cross validation datasets, k   
                            scoring='r2'   # evaluation metric 
                            ) 

In [492]:
cross_acc

array([0.90366651, 0.91236075, 0.91345015, 0.91749597, 0.90246243])

In [493]:
cross_acc.mean()   

0.9098871610626752

## Grid Search Tuning

In [494]:
pipeline_opt = make_pipeline(preprosessor, RandomForestRegressor())  #an unfitted model

In [495]:
pprint(pipeline.get_params())

{'columntransformer': ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                 ['holiday', 'workingday', 'season', 'weather',
                                  'windspeed', 'temp', 'humidity']),
                                ('time_extact',
                                 FunctionTransformer(func=<function extract at 0x7fa61f6e6af0>),
                                 ['datetime'])]),
 'columntransformer__do_nothing': 'passthrough',
 'columntransformer__n_jobs': None,
 'columntransformer__remainder': 'drop',
 'columntransformer__sparse_threshold': 0.3,
 'columntransformer__time_extact': FunctionTransformer(func=<function extract at 0x7fa61f6e6af0>),
 'columntransformer__time_extact__accept_sparse': False,
 'columntransformer__time_extact__check_inverse': True,
 'columntransformer__time_extact__func': <function extract at 0x7fa61f6e6af0>,
 'columntransformer__time_extact__inv_kw_args': None,
 'columntransformer__time_extact__inverse_func': None,
 

In [496]:
# define our hyperparameters to combine
hyperparam_grid = {
    'randomforestregressor__max_depth': [2, 5, 10, 20, 30],
    'randomforestregressor__n_estimators': [5, 10, 50, 100, 200],
    'randomforestregressor__min_samples_leaf': [1,5,10]
}


In [497]:
grid_cv = GridSearchCV(estimator=pipeline_opt,            # unfitted model/estimator
                       param_grid=hyperparam_grid,  # hyperparameters dict
                       cv=5,                        # number of folds, k
                       scoring='r2')                # scoring metric

In [498]:
# fit all models with all the different hyperparamters
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('do_nothing',
                                                                         'passthrough',
                                                                         ['holiday',
                                                                          'workingday',
                                                                          'season',
                                                                          'weather',
                                                                          'windspeed',
                                                                          'temp',
                                                                          'humidity']),
                                                                        ('time_extact',
                                                          

In [499]:
results_df = pd.DataFrame(grid_cv.cv_results_)

In [500]:
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.028994,0.003418,0.006879,0.000418,2,1,5,"{'randomforestregressor__max_depth': 2, 'rando...",0.246247,0.248602,0.250035,0.271167,0.211773,0.245565,0.019116,74
1,0.040394,0.000752,0.006704,0.000172,2,1,10,"{'randomforestregressor__max_depth': 2, 'rando...",0.283824,0.283506,0.252518,0.267878,0.272504,0.272046,0.011567,72
2,0.159409,0.000505,0.008589,0.000161,2,1,50,"{'randomforestregressor__max_depth': 2, 'rando...",0.294218,0.300291,0.264357,0.295691,0.270107,0.284933,0.014704,66
3,0.317814,0.011550,0.012630,0.001976,2,1,100,"{'randomforestregressor__max_depth': 2, 'rando...",0.295390,0.297913,0.263354,0.288876,0.278893,0.284885,0.012614,67
4,0.672970,0.049444,0.017005,0.001683,2,1,200,"{'randomforestregressor__max_depth': 2, 'rando...",0.292835,0.294829,0.268503,0.289536,0.277177,0.284576,0.010110,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.091454,0.000852,0.006987,0.000093,30,10,5,"{'randomforestregressor__max_depth': 30, 'rand...",0.803104,0.813399,0.820307,0.817684,0.805108,0.811920,0.006780,30
71,0.173181,0.001675,0.007951,0.000142,30,10,10,"{'randomforestregressor__max_depth': 30, 'rand...",0.798546,0.837685,0.835430,0.834759,0.815018,0.824288,0.015244,27
72,0.839746,0.012812,0.015588,0.000792,30,10,50,"{'randomforestregressor__max_depth': 30, 'rand...",0.810376,0.845350,0.836404,0.841804,0.830399,0.832867,0.012330,23
73,1.673775,0.029453,0.023951,0.000683,30,10,100,"{'randomforestregressor__max_depth': 30, 'rand...",0.810200,0.843958,0.841592,0.845194,0.828766,0.833942,0.013236,22


In [501]:
results_df.shape

(75, 16)

In [502]:
results_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_randomforestregressor__max_depth',
       'param_randomforestregressor__min_samples_leaf',
       'param_randomforestregressor__n_estimators', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')

In [503]:
results_df[['param_randomforestregressor__max_depth', 'param_randomforestregressor__min_samples_leaf', 'param_randomforestregressor__n_estimators', 'mean_test_score',
       'std_test_score', 'rank_test_score']].sort_values('rank_test_score')

,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__n_estimators,mean_test_score,std_test_score,rank_test_score
64,30,1,200,0.910432,0.006339,1
63,30,1,100,0.908613,0.006011,2
62,30,1,50,0.908392,0.004652,3
49,20,1,200,0.897152,0.006445,4
48,20,1,100,0.896824,0.006254,5
...,...,...,...,...,...,...
6,2,5,10,0.276570,0.015988,71
1,2,1,10,0.272046,0.011567,72
10,2,10,5,0.271503,0.016377,73
0,2,1,5,0.245565,0.019116,74


In [504]:
grid_cv.best_params_  

{'randomforestregressor__max_depth': 30,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__n_estimators': 200}

In [505]:
#estimating the test performance after tuning
pipeline_best = make_pipeline(preprosessor,RandomForestRegressor(max_depth=30, n_estimators=200, min_samples_leaf=1))

cross_acc = cross_val_score(estimator=pipeline_best,  # estimator: # the model you want to evaluate 
                            X=X_train,          # the training input data 
                            y=y_train,          # the training output data  
                            cv=5,               # number of cross validation datasets, k   
                            scoring='r2')       # evaluation metric 

In [506]:
cross_acc

array([0.90140163, 0.91152532, 0.91347642, 0.91941531, 0.90155798])

In [507]:
cross_acc.mean()

0.9094753311830374

In [508]:
pipeline_best.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                                  ['holiday', 'workingday',
                                                   'season', 'weather',
                                                   'windspeed', 'temp',
                                                   'humidity']),
                                                 ('time_extact',
                                                  FunctionTransformer(func=<function extract at 0x7fa61f6e6af0>),
                                                  ['datetime'])])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=30, n_estimators=200))])

In [509]:
pipeline_best.score(X_train, y_train)

0.9883585215366099

In [510]:
pipeline_best.score(X_test, y_test)
## After grid search tuning, R^2 score is improved to 89.88% from 88.09...

0.9268973365789074

## RMSLE

In [511]:
def rmsle(y_pred, y,convertExp=True):
    if convertExp:
        y_pred = np.exp(y_pred),
        y = np.exp(y)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y_pred]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [514]:
y_pred = pipeline_best.predict(X_train) 

In [515]:
rmsle(y_pred, y_train, convertExp=True)

#doesn't work

/var/folders/fs/bmr1lyws1dx3sp07l0gytvmc0000gn/T/ipykernel_81403/1284209182.py:3: RuntimeWarning: overflow encountered in exp
  y_pred = np.exp(y_pred),
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/fs/bmr1lyws1dx3sp07l0gytvmc0000gn/T/ipykernel_81403/1284209182.py:7: RuntimeWarning: overflow encountered in square
  calc = (log1 - log2) ** 2


inf

## Submission...

In [516]:
kaggle_data = pd.read_csv('test.csv')

In [517]:
kaggle_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014
...,...,...,...,...,...,...,...,...,...
6488,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014
6489,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014
6490,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014
6491,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981


In [518]:
predictions = pipeline_best.predict(kaggle_data)

In [519]:
submission = pd.DataFrame({'datetime':kaggle_data['datetime'],'count':predictions})

In [520]:
submission

,datetime,count
0,2011-01-20 00:00:00,17.651531
1,2011-01-20 01:00:00,13.240093
2,2011-01-20 02:00:00,11.226552
3,2011-01-20 03:00:00,11.114487
4,2011-01-20 04:00:00,8.765409
...,...,...
6488,2012-12-31 19:00:00,193.958833
6489,2012-12-31 20:00:00,141.603019
6490,2012-12-31 21:00:00,122.017389
6491,2012-12-31 22:00:00,110.755012


In [521]:
#Convert DataFrame to a csv file that can be uploaded
filename = 'Bike Sharing Demand RF.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)        

Saved file: Bike Sharing Demand RF.csv


## Result

## RMSLE on kaggle
Linear Regression: 9.15764
<br>
Random Forest: 0.51937
<br>
Random Forest(after grid search): 0.57039
<br>
Random Forest(features added and after grid seatch): 0.60666 Used both R^2 and MSLE but return same best hyperparameters
<br>
R^2 score of test data(splitted from train.csv) were improved but RMLSE on kaggle became worse...

In [ ]:


#END